# Step 2: Feature Engineering

Feature engineering combines the different data sources together to create a single data set of features (variables) that can be used to infer a machines's health condition over time. 

In this notebook, we will load the data stored in Azure Blob containers in the previous **Data Ingestion** notebook (`Code/data_ingestion.ipynb`). The note book uses several feature engineering methods to create a data set for use in our predictive maintenance machine learning solution.

In [1]:
# import the libraries
import os

import pyspark.sql.functions as F
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import datediff
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer

from pyspark.sql import SparkSession

from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

spark = SparkSession.builder.getOrCreate()

## Load data from Azure Blob storage container

We have previously downloaded and stored the following data in an Azure blob storage container:

  * Machines: Features differentiating each machine. For example age and model.
  * Error: The log of non-critical errors. These errors may still indicate an impending component failure.
  * Maint: Machine maintenance history detailing component replacement or regular maintenance activities withe the date of replacement.
  * Telemetry: The operating conditions of a machine e.g. data collected from sensors.
  * Failure history: The failure history of a machine or component within the machine.

We'll load these files from blob, and create our analysis data set here. We'll write this data set back into a new blob container to use in our model building and evaluation notebook later. 

Since the Azure Blob storage account name and account key are not passed between notebooks, you'll need to provide those here again.

In [ ]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "pdmamlworkbench"   ## "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "O5uLzNKX7o+ZHFXtHDyS87SIev9QHlkdX2IhIbxYwhRo7sA9zp45HOOFFttUp4r0LyWCcLQ0cCA7l+e8Ct3Yew==" ## "<account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Aquisition note book is stored in the dataingestion container.
CONTAINER_NAME = "dataingestion"

# Connect to your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

### Machines data set

Load the machines data set from your Azure blob.

In [ ]:
# create a local path where to store the results later.
LOCAL_DIRECT = 'dataingestion_mach_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'machines_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

machines = spark.read.parquet(LOCAL_DIRECT)

print(machines.count())
machines.toPandas().head(5)

### Errors data set

Load the errors data set from your Azure blob.

In [ ]:
# load the previous created final dataset into the workspace

# create a local path where to store the results later.
LOCAL_DIRECT = 'dataingestion_err_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'errors_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

errors = spark.read.parquet('dataingestion_err_result.parquet')

print(errors.count())
errors.toPandas().head(5)

### Maintenance data set

Load the maintenance data set from your Azure blob.

In [ ]:
# create a local path where to store the results later.
LOCAL_DIRECT = 'dataingestion_maint_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'maint_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

maint = spark.read.parquet('dataingestion_maint_result.parquet')

print(maint.count())
maint.toPandas().head(5)

### Telemetry

Load the telemetry data set from your Azure blob.

In [ ]:
# create a local path where to store the results later.
LOCAL_DIRECT = 'dataingestion_tel_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'telemetry_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

telemetry = spark.read.parquet('dataingestion_tel_result.parquet')

print(telemetry.count())
telemetry.toPandas().head(5)

### Failures data set

Load the failures data set from your Azure blob.

In [ ]:
# create a local path where to store the results later.
LOCAL_DIRECT = 'dataingestion_fail_result.parquet'
if not os.path.exists(LOCAL_DIRECT):
    os.makedirs(LOCAL_DIRECT)
    print('DONE creating a local directory!')

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'failure_files.parquet' in blob.name:
        local_file = os.path.join(LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

failures = spark.read.parquet('dataingestion_fail_result.parquet')

print(failures.count())
failures.toPandas().head(5)

## Feature engineering 

Feature engineering combines the different data sources together to create a single data set of features (variables) that can be used to infer a machines's health condition over time. The ultimate goal is to generate a single record for each time unit for each asset combining its features and labels to be fed into the machine learning algorithm. In order to prepare that clean final data set, some pre-processing steps should be taken. First step is to divide the duration of data collection into time units where each record belongs to a time unit for an asset.

The measurement unit for time can be in seconds, minutes, hours, days, months, cycles, miles or transactions depending on the efficiency of data preparation and the changes observed in the conditions of the asset from a time unit to the other or other factors specific to the domain. In other words, the time unit does not have to be the same as the frequency of data collection as in many cases data may not show any difference from one unit to the other. For example, if temperature values were being collected every 10 seconds, picking a time unit of 10 seconds for the whole analysis inflates the number of examples without providing any additional information. Better strategy would be to use average over an hour as an example.

### Rolling aggregates

For each record of an asset, we pick a rolling window of size "W" which is the number of units of time that we would like to compute historical aggregates for. We then compute rolling aggregate features using the W periods before the date of that record. Some example rolling aggregates can be rolling counts, means, standard deviations, outliers based on standard deviations, CUSUM measures, minimum and maximum values for the window. Another interesting technique is to capture trend changes, spikes and level changes using algorithms that detect anomalies in data using anomaly detection algorithms.

### Lag features
As mentioned earlier, in predictive maintenance, historical data usually comes with timestamps indicating the time of collection for each piece of data. There are many ways of creating features from the data that comes with timestamped data. In this section, we discuss some of these methods used for predictive maintenance. However, we are not limited by these methods alone. Since feature engineering is considered to be one of the most creative areas of predictive modeling, there could be many other ways to create features. Here, we provide some general techniques.

## Telemetry features

Because the telemetry data set is the largest time series data we have, we start feature engineering here. 

In [ ]:
# rolling mean
# Temporary storage for rolling means
tel_mean = telemetry

# Which features are we interested in telemetry data set
rolling_features = ['volt','rotate', 'pressure', 'vibration']
               
# We choose two windows for our rolling windows 3hrs, 24 hrs
lags = [3,24]

for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        tel_mean = tel_mean.withColumn(col_name+'_rollingmean_'+str(lag_n), F.avg(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

We repeat this rolling window process for the standard deviation. 

In [ ]:
# rolling standard deviation
# Temporary storage for rolling means
tel_sd = telemetry

for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        tel_sd = tel_sd.withColumn(col_name+'_rollingstd_'+str(lag_n), F.stddev(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

### Resample every 3 hours

In [28]:
# tel_mean rolling mean
# 3 hours = 10800 seconds  
time_val = 3 * 60 * 60

# I think this is grabbing datetime from the tel_sd data set, which is equivalent to telemetry
dt_truncated = ((round(unix_timestamp(col("datetime")) / time_val) * time_val).cast("timestamp"))

tel_mean_resampled = tel_mean.withColumn("dt_truncated", dt_truncated).drop('volt', 'rotate', 'pressure', 'vibration')

tel_mean_resampled1 = (tel_mean_resampled.groupBy("machineID","dt_truncated")
                               .agg(F.mean('volt_rollingmean_3').alias('volt_rollingmean_3'),
                                    F.mean('rotate_rollingmean_3').alias('rotate_rollingmean_3'), 
                                    F.mean('pressure_rollingmean_3').alias('pressure_rollingmean_3'), 
                                    F.mean('vibration_rollingmean_3').alias('vibration_rollingmean_3'), 
                                    F.mean('volt_rollingmean_24').alias('volt_rollingmean_24'),
                                    F.mean('rotate_rollingmean_24').alias('rotate_rollingmean_24'), 
                                    F.mean('pressure_rollingmean_24').alias('pressure_rollingmean_24'), 
                                    F.mean('vibration_rollingmean_24').alias('vibration_rollingmean_24')))

tel_mean_resampled1.where((col("machineID") == 1)).toPandas().head(5)
tel_mean_resampled1.count()

+--------------------+---------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+--------------------+
|            datetime|machineID|volt_rollingmean_3|rotate_rollingmean_3|pressure_rollingmean_3|vibration_rollingmean_3|volt_rollingmean_24|rotate_rollingmean_24|pressure_rollingmean_24|vibration_rollingmean_24|        dt_truncated|
+--------------------+---------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+--------------------+
|2015-01-01 06:00:...|        1|  151.919998705647|    530.813577555042|      101.788175260076|       49.6040134898504|   151.919998705647|     530.813577555042|       101.788175260076|        49.6040134898504|2015-01-01 06:00:...|
|2015-01-01 07:00:...|        1|  163.220999901059|    533.168554937213|

2921000

In [29]:
# tel_sd rolling sd
tel_sd_resampled = (tel_sd.withColumn("dt_truncated", dt_truncated).drop('volt', 'rotate', 'pressure', 'vibration')
                        .fillna(0))

tel_sd_resampled1 = (tel_sd_resampled.groupBy("machineID","dt_truncated")
                               .agg(F.sd('volt_rollingstd_3').alias('volt_rollingstd_3'),
                                    F.sd('rotate_rollingstd_3').alias('rotate_rollingstd_3'), 
                                    F.sd('pressure_rollingstd_3').alias('pressure_rollingstd_3'), 
                                    F.sd('vibration_rollingstd_3').alias('vibration_rollingstd_3'), 
                                    F.sd('volt_rollingstd_24').alias('volt_rollingstd_24'),
                                    F.sd('rotate_rollingstd_24').alias('rotate_rollingstd_24'), 
                                    F.sd('pressure_rollingstd_24').alias('pressure_rollingstd_24'), 
                                    F.sd('vibration_rollingstd_24').alias('vibration_rollingstd_24')))
tel_sd_resampled1.show(5)
tel_sd_resampled1.count()

+--------------------+---------+------------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+--------------------+
|            datetime|machineID| volt_rollingstd_3|rotate_rollingstd_3|pressure_rollingstd_3|vibration_rollingstd_3|volt_rollingstd_24|rotate_rollingstd_24|pressure_rollingstd_24|vibration_rollingstd_24|        dt_truncated|
+--------------------+---------+------------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+--------------------+
|2015-01-01 06:00:...|      148|               0.0|                0.0|                  0.0|                   0.0|               0.0|                 0.0|                   0.0|                    0.0|2015-01-01 06:00:...|
|2015-01-01 07:00:...|      148|19.182967495343476|  1.603237700069381|   1.4943454094246378|     7.

2921000

## Lag features from Errors

In [31]:
# create a column for each errorID 
error1 = errors.groupBy("machineID","datetime","errorID").pivot('errorID').agg(F.count('machineID').alias('dummy'))

# remove the column called errorID and fill in missing values
error2 = error1.drop('errorID').fillna(0)

# combine errors for a given machine in a given hour
error3 = (error2.groupBy("machineID","datetime")
                .agg(F.sum('error1').alias('error1sum'), 
                     F.sum('error2').alias('error2sum'), 
                     F.sum('error3').alias('error3sum'), 
                     F.sum('error4').alias('error4sum'), 
                     F.sum('error5').alias('error5sum')))

error3.toPandas().head(5)
error3.count(), len(error3.columns)

+---------+---------------------+-------+------+------+------+------+------+
|machineID|datetime             |errorID|error1|error2|error3|error4|error5|
+---------+---------------------+-------+------+------+------+------+------+
|14       |2015-04-01 13:00:00.0|error1 |1     |null  |null  |null  |null  |
|17       |2015-04-07 06:00:00.0|error5 |null  |null  |null  |null  |1     |
|29       |2015-07-22 06:00:00.0|error2 |null  |1     |null  |null  |null  |
|63       |2015-11-09 06:00:00.0|error1 |1     |null  |null  |null  |null  |
|71       |2015-03-11 06:00:00.0|error5 |null  |null  |null  |null  |1     |
+---------+---------------------+-------+------+------+------+------+------+
only showing top 5 rows



(11967, 8)

We want the same number of rows in error as in telemetry.



In [34]:
# join the telemetry data with errors
error_count = (telemetry.join(error3, ((telemetry['machineID'] == error3['machineID']) 
                                  & (telemetry['datetime'] == error3['datetime'])), "left")
               .drop('volt', 'rotate', 'pressure', 'vibration').drop(error3.machineID).drop(error3.datetime))

# fill in missing value
error_count1 = error_count.fillna(0)

error_count1.toPandas().show(5)
error_count1.count(), len(error_count1.columns)

+---------------------+---------+---------+---------+---------+---------+---------+
|datetime             |machineID|error1sum|error2sum|error3sum|error4sum|error5sum|
+---------------------+---------+---------+---------+---------+---------+---------+
|2015-01-01 06:00:00.0|1        |null     |null     |null     |null     |null     |
|2015-01-01 07:00:00.0|1        |null     |null     |null     |null     |null     |
|2015-01-01 08:00:00.0|1        |null     |null     |null     |null     |null     |
|2015-01-01 09:00:00.0|1        |null     |null     |null     |null     |null     |
|2015-01-01 10:00:00.0|1        |null     |null     |null     |null     |null     |
+---------------------+---------+---------+---------+---------+---------+---------+
only showing top 5 rows



(8761000, 7)

In [38]:
rolling_features1 = ['error1sum','error2sum', 'error3sum', 'error4sum', 'error5sum']
               
# lag window 24 hrs
lags = [24]

# rolling mean
err_mean = error_count1

for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features1:
        err_mean = err_mean.withColumn(col_name+'_rollingmean_'+str(lag_n), F.avg(col(col_name)).over(wSpec))
        print("Lag = %d, Column = %s" % (lag_n, col_name))

Lag = 24, Column = error1sum
Lag = 24, Column = error2sum
Lag = 24, Column = error3sum
Lag = 24, Column = error4sum
Lag = 24, Column = error5sum


### Resample to every 3 hours

In [40]:
dt_truncated = ((round(unix_timestamp(col("datetime")) / time_val) * time_val)
    .cast("timestamp"))

err_mean_resampled = (err_mean.withColumn("dt_truncated", dt_truncated)
                    .drop('error1sum', 'error2sum', 'error3sum', 'error4sum', 'error5sum').fillna(0))

err_mean_resampled1 = (err_mean_resampled.groupBy("machineID","dt_truncated")
                               .agg(F.mean('error1sum_rollingmean_24').alias('error1sum_rollingmean_24'), 
                                    F.mean('error2sum_rollingmean_24').alias('error2sum_rollingmean_24'), 
                                    F.mean('error3sum_rollingmean_24').alias('error3sum_rollingmean_24'), 
                                    F.mean('error4sum_rollingmean_24').alias('error4sum_rollingmean_24'), 
                                    F.mean('error5sum_rollingmean_24').alias('error5sum_rollingmean_24')))
err_mean_resampled1.toPandas().head(5)
err_mean_resampled1.count()

+--------------------+---------+------------------------+------------------------+------------------------+------------------------+------------------------+--------------------+
|            datetime|machineID|error1sum_rollingmean_24|error2sum_rollingmean_24|error3sum_rollingmean_24|error4sum_rollingmean_24|error5sum_rollingmean_24|        dt_truncated|
+--------------------+---------+------------------------+------------------------+------------------------+------------------------+------------------------+--------------------+
|2015-01-01 06:00:...|      148|                     0.0|                     0.0|                     0.0|                     0.0|                     0.0|2015-01-01 06:00:...|
|2015-01-01 07:00:...|      148|                     0.0|                     0.0|                     0.0|                     0.0|                     0.0|2015-01-01 06:00:...|
|2015-01-01 08:00:...|      148|                     0.0|                     0.0|                     0.

2921000

## Days since last replacement from maintenance 

In [42]:
# create a column for each comp 
maint1 = maint.groupBy("machineID","datetime","comp").pivot('comp').agg(F.count('machineID').alias('dummy'))

# remove the column called comp and fill in missing values
maint2 = maint1.drop('comp').fillna(0)

# combine maintenance for a given machine in a given hour
maint3 = (maint2.groupBy("machineID","datetime").agg(F.sum('comp1').alias('comp1sum'), 
                                                    F.sum('comp2').alias('comp2sum'), 
                                                    F.sum('comp3').alias('comp3sum'),
                                                    F.sum('comp4').alias('comp4sum')))
maint3.toPandas().head(5)

+---------+---------------------+-----+-----+-----+-----+-----+
|machineID|datetime             |comp |comp1|comp2|comp3|comp4|
+---------+---------------------+-----+-----+-----+-----+-----+
|8        |2015-07-03 06:00:00.0|comp3|null |null |1    |null |
|8        |2015-08-02 06:00:00.0|comp2|null |1    |null |null |
|24       |2015-04-20 06:00:00.0|comp3|null |null |1    |null |
|35       |2015-05-05 06:00:00.0|comp2|null |1    |null |null |
|38       |2015-04-02 06:00:00.0|comp4|null |null |null |1    |
+---------+---------------------+-----+-----+-----+-----+-----+
only showing top 5 rows



(32592, 8)

## Days since last replacement for component-1

In [49]:
test_maint_comp1 = (maint3.where((col("comp1sum") == '1')).withColumnRenamed('datetime','datetime_maint')
                           .drop('comp2sum', 'comp3sum', 'comp4sum'))

test_tel_comp1 = (telemetry.withColumnRenamed('datetime','datetime_tel')
                  .drop(telemetry.volt).drop(telemetry.rotate).drop(telemetry.pressure).drop(telemetry.vibration))

test_maint_tel_comp1 = test_tel_comp1.join(test_maint_comp1, ((test_tel_comp1['machineID']==
                                                               test_maint_comp1['machineID']) 
                                            & (test_tel_comp1['datetime_tel'] > test_maint_comp1['datetime_maint']) 
                                            & (test_maint_comp1['comp1sum'] == '1'))).drop(test_maint_comp1.machineID)

comp1 = (test_maint_tel_comp1.withColumn("sincelastcomp1", 
              datediff(test_maint_tel_comp1.datetime_tel, test_maint_tel_comp1.datetime_maint))
              .drop(test_maint_tel_comp1.datetime_maint).drop(test_maint_tel_comp1.comp1sum))

comp1.toPandas().head(5)

+--------------------+---------+--------------+
|        datetime_tel|machineID|sincelastcomp1|
+--------------------+---------+--------------+
|2015-01-01 06:00:...|        1|           109|
|2015-01-01 07:00:...|        1|           109|
|2015-01-01 08:00:...|        1|           109|
|2015-01-01 09:00:...|        1|           109|
|2015-01-01 10:00:...|        1|           109|
+--------------------+---------+--------------+
only showing top 5 rows



## Days since last replacement for component-2

In [54]:
test_maint_comp2 = (maint3.where(col("comp2sum") == '1').withColumnRenamed('datetime','datetime_maint')
                         .drop('comp1sum', 'comp3sum', 'comp4sum'))

test_tel_comp2 = (telemetry.withColumnRenamed('datetime','datetime_tel')
                          .drop(telemetry.volt).drop(telemetry.rotate).drop(telemetry.pressure)
                          .drop(telemetry.vibration))

test_maint_tel_comp2 = (test_tel_comp2.join(test_maint_comp2, ((test_tel_comp2['machineID']==
                                                                test_maint_comp2['machineID']) 
                                        & (test_tel_comp2['datetime_tel'] > test_maint_comp2['datetime_maint']) 
                                        & (test_maint_comp2['comp2sum'] == '1') 
                                           )).drop(test_maint_comp2.machineID))

comp2 = (test_maint_tel_comp2.withColumn("sincelastcomp2", 
              datediff(test_maint_tel_comp2.datetime_tel, test_maint_tel_comp2.datetime_maint))
              .drop(test_maint_tel_comp2.datetime_maint).drop(test_maint_tel_comp2.comp2sum))
comp2.toPandas().head(5)

+--------------------+---------+--------------+
|        datetime_tel|machineID|sincelastcomp2|
+--------------------+---------+--------------+
|2015-01-01 06:00:...|        1|           109|
|2015-01-01 07:00:...|        1|           109|
|2015-01-01 08:00:...|        1|           109|
|2015-01-01 09:00:...|        1|           109|
|2015-01-01 10:00:...|        1|           109|
+--------------------+---------+--------------+
only showing top 5 rows



## Days since last replacement for component-3

In [59]:
test_maint_comp3 = (maint3.where(col("comp3sum") == '1').withColumnRenamed('datetime','datetime_maint')
                          .drop('comp1sum', 'comp2sum', 'comp4sum'))

test_tel_comp3 = (telemetry.withColumnRenamed('datetime','datetime_tel')
                    .drop(telemetry.volt).drop(telemetry.rotate).drop(telemetry.pressure).drop(telemetry.vibration))

test_maint_tel_comp3 = test_tel_comp3.join(test_maint_comp3, ((test_tel_comp3['machineID']==
                                                               test_maint_comp3['machineID']) 
                                        & (test_tel_comp3['datetime_tel'] > test_maint_comp3['datetime_maint']) 
                                        & (test_maint_comp3['comp3sum'] == '1') 
                                           )).drop(test_maint_comp3.machineID)

comp3 = (test_maint_tel_comp3.withColumn("sincelastcomp3", 
              datediff(test_maint_tel_comp3.datetime_tel, test_maint_tel_comp3.datetime_maint))
              .drop(test_maint_tel_comp3.datetime_maint).drop(test_maint_tel_comp3.comp3sum))
comp3.toPandas().head(5)

+--------------------+---------+--------------+
|        datetime_tel|machineID|sincelastcomp3|
+--------------------+---------+--------------+
|2015-01-01 06:00:...|        1|            49|
|2015-01-01 07:00:...|        1|            49|
|2015-01-01 08:00:...|        1|            49|
|2015-01-01 09:00:...|        1|            49|
|2015-01-01 10:00:...|        1|            49|
+--------------------+---------+--------------+
only showing top 5 rows



## Days since last replacement for component-4

In [64]:
test_maint_comp4 = (maint3.where(col("comp4sum") == '1').withColumnRenamed('datetime','datetime_maint')
                         .drop('comp1sum', 'comp2sum', 'comp3sum'))

test_tel_comp4 = (telemetry.withColumnRenamed('datetime','datetime_tel')
                  .drop(telemetry.volt).drop(telemetry.rotate).drop(telemetry.pressure).drop(telemetry.vibration))

test_maint_tel_comp4 = test_tel_comp4.join(test_maint_comp4, ((test_tel_comp4['machineID']==
                                                               test_maint_comp4['machineID']) 
                                        & (test_tel_comp4['datetime_tel'] > test_maint_comp4['datetime_maint']) 
                                        & (test_maint_comp4['comp4sum'] == '1'))).drop(test_maint_comp4.machineID)

comp4 = (test_maint_tel_comp4.withColumn("sincelastcomp4", 
              datediff(test_maint_tel_comp4.datetime_tel, test_maint_tel_comp4.datetime_maint))
              .drop(test_maint_tel_comp4.datetime_maint).drop(test_maint_tel_comp4.comp4sum))
comp4.toPandas().head(5)

+--------------------+---------+--------------+
|        datetime_tel|machineID|sincelastcomp4|
+--------------------+---------+--------------+
|2015-01-01 06:00:...|        1|           184|
|2015-01-01 07:00:...|        1|           184|
|2015-01-01 08:00:...|        1|           184|
|2015-01-01 09:00:...|        1|           184|
|2015-01-01 10:00:...|        1|           184|
+--------------------+---------+--------------+
only showing top 5 rows



##  Combine comp1, comp2, comp3, comp4 to generate the maintenance feature set

In [69]:
# left join comp1 with (comp2, comp3, comp4) 
# left join comp2 with (comp3, comp4) 
# left join comp3, comp4 
comp3_4 = (comp3.join(comp4, ((comp3['machineID'] == comp4['machineID']) 
                                  & (comp3['datetime_tel'] == comp4['datetime_tel'])), "left")
                                  .drop(comp4.machineID).drop(comp4.datetime_tel))comp2_3_4 = (comp2.join(comp3_4, ((comp2['machineID'] == comp3_4['machineID']) 
                                  & (comp2['datetime_tel'] == comp3_4['datetime_tel'])), "left")
                                  .drop(comp3_4.machineID).drop(comp3_4.datetime_tel))
comp1_2_3_4 = (comp1.join(comp2_3_4, ((comp1['machineID'] == comp2_3_4['machineID']) 
                                  & (comp1['datetime_tel'] == comp2_3_4['datetime_tel'])), "left")
                                 .drop(comp2_3_4.machineID).drop(comp2_3_4.datetime_tel))
comp1_2_3_4_final = (comp1_2_3_4.groupBy("machineID", "datetime_tel")
                                .agg(F.max('sincelastcomp1').alias('sincelastcomp1'), 
                                     F.max('sincelastcomp2').alias('sincelastcomp2'), 
                                     F.max('sincelastcomp3').alias('sincelastcomp3'), 
                                     F.max('sincelastcomp4').alias('sincelastcomp4')))

# fill in missing value
maint_count1 = comp1_2_3_4_final.fillna(0)

maint_count1.toPandas().head(5)

### Resample to every 3 hours

In [72]:
# maint_count1 maintenance 
dt_truncated = ((round(unix_timestamp(col("datetime_tel")) / time_val) * time_val)
    .cast("timestamp"))

maint_resampled = maint_count1.withColumn("dt_truncated", dt_truncated)
maint_resampled1 = (maint_resampled.groupBy("machineID","dt_truncated")
                                  .agg(F.mean('sincelastcomp1').alias('comp1sum'), 
                                       F.mean('sincelastcomp2').alias('comp2sum'), 
                                       F.mean('sincelastcomp3').alias('comp3sum'), 
                                       F.mean('sincelastcomp4').alias('comp4sum')))
maint_resampled1.toPandas().head(5)

+---------+--------------------+--------------+--------------+--------------+--------------+--------------------+
|machineID|        datetime_tel|sincelastcomp1|sincelastcomp2|sincelastcomp3|sincelastcomp4|        dt_truncated|
+---------+--------------------+--------------+--------------+--------------+--------------+--------------------+
|        1|2015-01-01 06:00:...|           109|           109|            49|           184|2015-01-01 06:00:...|
|        1|2015-01-05 10:00:...|           113|           113|            53|           188|2015-01-05 09:00:...|
|        1|2015-01-10 18:00:...|           118|           118|            58|           193|2015-01-10 18:00:...|
|        1|2015-01-10 20:00:...|           118|           118|            58|           193|2015-01-10 21:00:...|
|        1|2015-01-11 13:00:...|           119|           119|            59|           194|2015-01-11 12:00:...|
+---------+--------------------+--------------+--------------+--------------+-----------

[('machineID', 'int'),
 ('datetime_tel', 'timestamp'),
 ('sincelastcomp1', 'int'),
 ('sincelastcomp2', 'int'),
 ('sincelastcomp3', 'int'),
 ('sincelastcomp4', 'int'),
 ('dt_truncated', 'timestamp')]

## Machine features - need to do one hot encoding for variable model 

In [75]:
# one hot encoding of the variable model
catVarNames = ['model']  
    
sIndexers = [StringIndexer(inputCol=x, outputCol=x + '_indexed') for x in catVarNames]

machines_cat = Pipeline(stages=sIndexers).fit(machines).transform(machines)

# one-hot encode
ohEncoders = [OneHotEncoder(inputCol=x + '_indexed', outputCol=x + '_encoded')
              for x in catVarNames]
ohPipelineModel = Pipeline(stages=ohEncoders).fit(machines_cat)
machines_cat = ohPipelineModel.transform(machines_cat)

drop_list = [col_n for col_n in machines_cat.columns if 'indexed' in col_n]

machines_edit = machines_cat.select([column for column in machines_cat.columns if column not in drop_list])

machines_edit.toPandas().head(5)

+---------+------+---+-------------+
|machineID| model|age|model_encoded|
+---------+------+---+-------------+
|        1|model2| 18|(3,[2],[1.0])|
|        2|model4|  7|(3,[1],[1.0])|
|        3|model3|  8|(3,[0],[1.0])|
|        4|model3|  7|(3,[0],[1.0])|
|        5|model2|  2|(3,[2],[1.0])|
+---------+------+---+-------------+
only showing top 5 rows



# Creating final feature matrix

In [76]:
# join error with components
#err_mean_resampled1.show(3)
#maint_resampled1.show(3)

error_maint = (err_mean_resampled1.join(maint_resampled1, 
                                ((err_mean_resampled1['machineID'] == maint_resampled1['machineID']) 
                                  & (err_mean_resampled1['dt_truncated'] == maint_resampled1['dt_truncated'])), "left")
                                  .drop(maint_resampled1.machineID).drop(maint_resampled1.dt_truncated))
#error_maint.show(10, False)
#error_maint.count(), len(error_maint.columns)

# now join with machines
#machines_edit.show(1)

err_maint_mach = (error_maint.join(machines_edit, ((error_maint['machineID'] == machines_edit['machineID'])), "left")
                             .drop(machines_edit.machineID))
err_maint_mach_select = (err_maint_mach.select([c for c in err_maint_mach.columns if c not in 
                                               {'error1sum', 'error2sum', 'error3sum', 'error4sum', 'error5sum'}]))
#err_maint_mach_select.show(10, False)
#err_maint_mach_select.count(), len(err_maint_mach_select.columns)

telemetry_all = (tel_mean_resampled1.join(tel_sd_resampled1, 
                             ((tel_mean_resampled1['machineID'] == tel_sd_resampled1['machineID']) 
                              & (tel_mean_resampled1['dt_truncated'] == tel_sd_resampled1['dt_truncated'])), "left")
                              .drop(tel_sd_resampled1.machineID).drop(tel_sd_resampled1.dt_truncated))
#telemetry_all.show(10, False)
#telemetry_all.count(), len(telemetry_all.columns)

# join telemetry_all with err_maint_mach_select to create final feature matrix
final_feat = (telemetry_all.join(err_maint_mach_select, 
                                ((telemetry_all['machineID'] == err_maint_mach_select['machineID']) 
                                  & (telemetry_all['dt_truncated'] == err_maint_mach_select['dt_truncated'])), "left")
                                 .drop(err_maint_mach_select.machineID).drop(err_maint_mach_select.dt_truncated))
final_feat.show(5, False)
#final_feat.count(), len(final_feat.columns)

+---------+---------------------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+------------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+------------------------+------------------------+------------------------+------------------------+------------------------+--------+--------+--------+--------+------+---+-------------+
|machineID|dt_truncated         |volt_rollingmean_3|rotate_rollingmean_3|pressure_rollingmean_3|vibration_rollingmean_3|volt_rollingmean_24|rotate_rollingmean_24|pressure_rollingmean_24|vibration_rollingmean_24|volt_rollingstd_3 |rotate_rollingstd_3|pressure_rollingstd_3|vibration_rollingstd_3|volt_rollingstd_24|rotate_rollingstd_24|pressure_rollingstd_24|vibration_rollingstd_24|error1sum_rollingmean_24|error2sum_rollingmean_24

# Label construction

In [77]:
# check failure sample data
failures.show(5)

# check the dimensions of the data
failures.count(), len(failures.columns)

+--------------------+---------+-------+
|            datetime|machineID|failure|
+--------------------+---------+-------+
|2015-02-04 06:00:...|        1|  comp3|
|2015-03-21 06:00:...|        1|  comp1|
|2015-04-05 06:00:...|        1|  comp4|
|2015-05-05 06:00:...|        1|  comp3|
|2015-05-20 06:00:...|        1|  comp2|
+--------------------+---------+-------+
only showing top 5 rows



(6726, 3)

In [78]:
# check to see if there are duplicate rows based on machine, datetime
failures1 = failures.dropDuplicates(['machineID', 'datetime'])

# check the dimensions of the data
failures1.count(), len(failures1.columns)

(6368, 3)

In [79]:
# map the failure data to final feature matrix

labeled_features = (final_feat.join(failures1, ((final_feat['machineID'] == failures1['machineID']) 
                                  & (final_feat['dt_truncated'] == failures1['datetime'])), "left")
                                  .drop(failures1.machineID).drop(failures1.datetime))
labeled_features.show(5, False)
#labeled_features.count(), len(labeled_features.columns)

+---------+---------------------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+------------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+------------------------+------------------------+------------------------+------------------------+------------------------+--------+--------+--------+--------+------+---+-------------+-------+
|machineID|dt_truncated         |volt_rollingmean_3|rotate_rollingmean_3|pressure_rollingmean_3|vibration_rollingmean_3|volt_rollingmean_24|rotate_rollingmean_24|pressure_rollingmean_24|vibration_rollingmean_24|volt_rollingstd_3 |rotate_rollingstd_3|pressure_rollingstd_3|vibration_rollingstd_3|volt_rollingstd_24|rotate_rollingstd_24|pressure_rollingstd_24|vibration_rollingstd_24|error1sum_rollingmean_24|error2sum_rollin

In [80]:
# recoding the column 'failure' to be numeric double for the pyspark classification models
labeled_features1 = (labeled_features.withColumn('failure', F.when(col('failure') == "comp1", 1.0)
                                     .otherwise(col('failure')))
                                     .withColumn('failure', F.when(col('failure') == "comp2", 2.0)
                                     .otherwise(col('failure')))
                                     .withColumn('failure', F.when(col('failure') == "comp3", 3.0)
                                     .otherwise(col('failure')))
                                     .withColumn('failure', F.when(col('failure') == "comp4", 4.0)
                                     .otherwise(col('failure'))))

labeled_features2 = labeled_features1.withColumn("failure1", labeled_features1["failure"].cast(DoubleType()))

#labeled_features2.groupBy('failure').count().show()
#labeled_features2.groupBy('failure1').count().show()

In [81]:
# check data schema
labeled_features2.dtypes

[('machineID', 'int'),
 ('dt_truncated', 'timestamp'),
 ('volt_rollingmean_3', 'double'),
 ('rotate_rollingmean_3', 'double'),
 ('pressure_rollingmean_3', 'double'),
 ('vibration_rollingmean_3', 'double'),
 ('volt_rollingmean_24', 'double'),
 ('rotate_rollingmean_24', 'double'),
 ('pressure_rollingmean_24', 'double'),
 ('vibration_rollingmean_24', 'double'),
 ('volt_rollingstd_3', 'double'),
 ('rotate_rollingstd_3', 'double'),
 ('pressure_rollingstd_3', 'double'),
 ('vibration_rollingstd_3', 'double'),
 ('volt_rollingstd_24', 'double'),
 ('rotate_rollingstd_24', 'double'),
 ('pressure_rollingstd_24', 'double'),
 ('vibration_rollingstd_24', 'double'),
 ('error1sum_rollingmean_24', 'double'),
 ('error2sum_rollingmean_24', 'double'),
 ('error3sum_rollingmean_24', 'double'),
 ('error4sum_rollingmean_24', 'double'),
 ('error5sum_rollingmean_24', 'double'),
 ('comp1sum', 'double'),
 ('comp2sum', 'double'),
 ('comp3sum', 'double'),
 ('comp4sum', 'double'),
 ('model', 'string'),
 ('age', 'int'

In [82]:
labeled_features3 = labeled_features2.drop('failure').fillna(0)
labeled_features3.dtypes
#labeled_features3.groupBy('failure1').count().show()

[('machineID', 'int'),
 ('dt_truncated', 'timestamp'),
 ('volt_rollingmean_3', 'double'),
 ('rotate_rollingmean_3', 'double'),
 ('pressure_rollingmean_3', 'double'),
 ('vibration_rollingmean_3', 'double'),
 ('volt_rollingmean_24', 'double'),
 ('rotate_rollingmean_24', 'double'),
 ('pressure_rollingmean_24', 'double'),
 ('vibration_rollingmean_24', 'double'),
 ('volt_rollingstd_3', 'double'),
 ('rotate_rollingstd_3', 'double'),
 ('pressure_rollingstd_3', 'double'),
 ('vibration_rollingstd_3', 'double'),
 ('volt_rollingstd_24', 'double'),
 ('rotate_rollingstd_24', 'double'),
 ('pressure_rollingstd_24', 'double'),
 ('vibration_rollingstd_24', 'double'),
 ('error1sum_rollingmean_24', 'double'),
 ('error2sum_rollingmean_24', 'double'),
 ('error3sum_rollingmean_24', 'double'),
 ('error4sum_rollingmean_24', 'double'),
 ('error5sum_rollingmean_24', 'double'),
 ('comp1sum', 'double'),
 ('comp2sum', 'double'),
 ('comp3sum', 'double'),
 ('comp4sum', 'double'),
 ('model', 'string'),
 ('age', 'int'

In [83]:
# build the code for backfill with all machine data
label_bfill1 = labeled_features3
label_bfill1.show(1)

+---------+--------------------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+-----------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+------------------------+------------------------+------------------------+------------------------+------------------------+--------+--------+--------+--------+------+---+-------------+--------+
|machineID|        dt_truncated|volt_rollingmean_3|rotate_rollingmean_3|pressure_rollingmean_3|vibration_rollingmean_3|volt_rollingmean_24|rotate_rollingmean_24|pressure_rollingmean_24|vibration_rollingmean_24|volt_rollingstd_3|rotate_rollingstd_3|pressure_rollingstd_3|vibration_rollingstd_3|volt_rollingstd_24|rotate_rollingstd_24|pressure_rollingstd_24|vibration_rollingstd_24|error1sum_rollingmean_24|error2sum_rollingme

In [84]:
# lag values to manually backfill label (bfill =7)
my_window = Window.partitionBy('machineID').orderBy(label_bfill1.dt_truncated.desc())

label_bfill1 = label_bfill1.withColumn("prev_value1", F.lag(label_bfill1.failure1).over(my_window)).fillna(0)
label_bfill1 = label_bfill1.withColumn("prev_value2", F.lag(label_bfill1.prev_value1).over(my_window)).fillna(0) 
label_bfill1 = label_bfill1.withColumn("prev_value3", F.lag(label_bfill1.prev_value2).over(my_window)).fillna(0) 
label_bfill1 = label_bfill1.withColumn("prev_value4", F.lag(label_bfill1.prev_value3).over(my_window)).fillna(0) 
label_bfill1 = label_bfill1.withColumn("prev_value5", F.lag(label_bfill1.prev_value4).over(my_window)).fillna(0) 
label_bfill1 = label_bfill1.withColumn("prev_value6", F.lag(label_bfill1.prev_value5).over(my_window)).fillna(0) 
label_bfill1 = label_bfill1.withColumn("prev_value7", F.lag(label_bfill1.prev_value6).over(my_window)).fillna(0)

In [85]:
# create the label column 
label_bfill2 = (label_bfill1.withColumn('label', label_bfill1.failure1 + label_bfill1.prev_value1 
                         + label_bfill1.prev_value2 + label_bfill1.prev_value3 + label_bfill1.prev_value4 
                         + label_bfill1.prev_value5 + label_bfill1.prev_value6 + label_bfill1.prev_value7))
label_bfill2 = label_bfill2.withColumn('label_e', F.when(col('label') > 4, 4.0).otherwise(col('label')))

In [86]:
label_bfill3 = (label_bfill2.drop(label_bfill2.prev_value1).drop(label_bfill2.prev_value2)
              .drop(label_bfill2.prev_value3).drop(label_bfill2.prev_value4)
              .drop(label_bfill2.prev_value5).drop(label_bfill2.prev_value6)
              .drop(label_bfill2.prev_value7).drop(label_bfill2.label))

In [87]:
label_bfill3.show(1)

+---------+--------------------+------------------+--------------------+----------------------+-----------------------+-------------------+---------------------+-----------------------+------------------------+-----------------+-------------------+---------------------+----------------------+------------------+--------------------+----------------------+-----------------------+------------------------+------------------------+------------------------+------------------------+------------------------+--------+--------+--------+--------+------+---+-------------+--------+-------+
|machineID|        dt_truncated|volt_rollingmean_3|rotate_rollingmean_3|pressure_rollingmean_3|vibration_rollingmean_3|volt_rollingmean_24|rotate_rollingmean_24|pressure_rollingmean_24|vibration_rollingmean_24|volt_rollingstd_3|rotate_rollingstd_3|pressure_rollingstd_3|vibration_rollingstd_3|volt_rollingstd_24|rotate_rollingstd_24|pressure_rollingstd_24|vibration_rollingstd_24|error1sum_rollingmean_24|error2sum_r

In [88]:
# write the final result as parquet file in blob location 
# https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
CONTAINER_NAME = "featureengineering"

# Create a new container if necessary, otherwise you can use an existing container.
# This command creates the container if it does not already exist. Else it does nothing.
my_service.create_container(CONTAINER_NAME, 
                            fail_on_exist=False, 
                            public_access=PublicAccess.Container)

# you decide to partition the dataframe into three files and save them in the current folder.
# if you wish to visualize them in the run history Output Files, specify the path 
# as './outputs/multiple_files.parquet'.
#label_bfill3.coalesce(3).write.mode('overwrite').parquet('multiple_files.parquet')
label_bfill3.write.mode('overwrite').parquet('featureengineering_files.parquet')

# unlike the single file case, for multiple files we need to first delete results from the 
# previous run before uploading.
for blob in my_service.list_blobs(CONTAINER_NAME):
    if 'featureengineering_files.parquet' in blob.name:
        my_service.delete_blob(CONTAINER_NAME, blob.name)

# upload the entire folder into blob storage
for name in glob.iglob('featureengineering_files.parquet/*'):
    print(os.path.abspath(name))
    my_service.create_blob_from_path(CONTAINER_NAME, name, name)

print("Feature engineering final dataset files saved!")

NameError: name 'glob' is not defined

('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
